In [17]:
import csv
import json
import subprocess

import numpy as np
import MeCab
from sklearn import feature_extraction

dicdir = subprocess.check_output(['mecab-config', '--dicdir']).decode().strip()
m = MeCab.Tagger(' -d {}/mecab-ipadic-neologd'.format(dicdir))

In [18]:
def load_json(path):
    with open(path, "r") as f:
        json_dict = json.load(f)
    return json_dict

In [31]:
def nodes(text):
    node = m.parseToNode(text)
    while node:
        # XXX: Monkey Patching
        node.features = node.feature.split(',')

        if node.features[0] != 'BOS/EOS':
            yield node

        node = node.next


def wakachi(text):
    def wanted(node):
        return node.features[0] not in (
            '助詞',
            '助動詞',
            '接続詞',
            '動詞',
            '記号',
        )

    words = [n.surface for n in nodes(text) if wanted(n)]
    return ' '.join(words)

In [32]:
def set_stopwords():
    stop_word_file_path = "nlp/stopword_japanese.txt"
    with open(stop_word_file_path, "r") as f:
        stopwords = [line.strip() for line in f]
        stopwords = [ss for ss in stopwords if not ss == u'']
    return stopwords

In [33]:
def join_list_str(list):
    return ' '.join(list)

def exclude_stopword(text, stopwords):
    changed_text = [token for token in text.lower().split(" ") if token != "" if token not in stopwords]
    changed_text = join_list_str(changed_text)
    return changed_text

In [34]:
def main():
    review_json_path = "../data/coco_reviews.json"
    json_dict = load_json(review_json_path)
    reviews = []
    stopwords = set_stopwords()
    for i in range(1, len(json_dict)+1):
        # print(json_dict[str(i)]["title"])
        # print(json_dict[str(i)]["reviews"])
        all_review_str = join_list_str(
            [exclude_stopword(wakachi(review_info["review"]), stopwords) for review_info in json_dict[str(i)]["reviews"]]
        )
        reviews += [all_review_str]
    # max_dfとmin_dfによって結果は大きく変わる
    vectorizer = feature_extraction.text.TfidfVectorizer(
        decode_error="ignore",
        strip_accents="unicode",
        lowercase=True,
        analyzer="word",
        max_df=0.5,
        min_df=0.3
    )
    
    X = vectorizer.fit_transform(reviews).toarray()
    index = X.argsort(axis=1)[:, ::-1]
    feature_names = np.array(vectorizer.get_feature_names())
    feature_words = feature_names[index]
    
    # FIXME: functionize me
    with open("../data/nlp/tfidf.csv", "w") as f:
        writer = csv.writer(f, lineterminator="\n")
        writer.writerows(feature_words)
    return feature_words

main()

array([['最後', 'もう', '演技', ..., '映像', '展開', 'キャスト'],
       ['面白かっ', 'よかっ', '印象', ..., '意味', '役者', 'こんな'],
       ['面白かっ', 'よかっ', '印象', ..., '意味', '役者', 'こんな'],
       ...,
       ['演出', '意味', '演技', ..., '人間', '俳優', '最高'],
       ['面白い', '原作', '描写', ..., 'とう', 'ラスト', '人間'],
       ['原作', 'キャスト', '面白かっ', ..., '最高', 'ちょっと', 'こんな']], dtype='<U5')